# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 8 2022 2:27PM,243689,20,ALU0007472,Alumier Labs Inc.,Released
1,Aug 8 2022 2:27PM,243689,20,ALUR010214888,Alumier Labs Inc.,Released
2,Aug 8 2022 2:27PM,243689,20,ALUR182462799,Alumier Labs Inc.,Released
3,Aug 8 2022 2:27PM,243689,20,ALUR210913401,Alumier Labs Inc.,Released
4,Aug 8 2022 2:27PM,243689,20,ALUR368311502,Alumier Labs Inc.,Released
5,Aug 8 2022 2:27PM,243689,20,ALUR392191570,Alumier Labs Inc.,Released
6,Aug 8 2022 2:27PM,243689,20,ALUR443802689,Alumier Labs Inc.,Released
7,Aug 8 2022 2:27PM,243689,20,ALUR448225620,Alumier Labs Inc.,Released
8,Aug 8 2022 2:27PM,243689,20,ALUR496048783,Alumier Labs Inc.,Released
9,Aug 8 2022 2:27PM,243689,20,ALUR615880248,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,243687,Released,15
22,243688,Released,9
23,243689,Released,34
24,243690,Released,1
25,243692,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243687,NaN,15.0
243688,NaN,9.0
243689,NaN,34.0
243690,NaN,1.0
243692,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243608,2.0,0.0
243612,1.0,0.0
243617,0.0,1.0
243627,0.0,1.0
243661,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243608,2,0
243612,1,0
243617,0,1
243627,0,1
243661,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243608,2,0
1,243612,1,0
2,243617,0,1
3,243627,0,1
4,243661,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243608,2,
1,243612,1,
2,243617,,1
3,243627,,1
4,243661,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 8 2022 2:27PM,243689,20,Alumier Labs Inc.
34,Aug 8 2022 2:26PM,243692,19,"AdvaGen Pharma, Ltd"
37,Aug 8 2022 2:24PM,243687,20,Alumier Labs Inc.
52,Aug 8 2022 2:20PM,243690,10,ISDIN Corporation
53,Aug 8 2022 2:17PM,243688,10,ISDIN Corporation
62,Aug 8 2022 2:13PM,243686,19,"NAPP Technologies, LLC"
63,Aug 8 2022 2:06PM,243685,20,"Exact-Rx, Inc."
67,Aug 8 2022 2:06PM,243684,20,"Exact-Rx, Inc."
84,Aug 8 2022 1:53PM,243683,19,"AdvaGen Pharma, Ltd"
85,Aug 8 2022 1:41PM,243682,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 8 2022 2:27PM,243689,20,Alumier Labs Inc.,,34
1,Aug 8 2022 2:26PM,243692,19,"AdvaGen Pharma, Ltd",,3
2,Aug 8 2022 2:24PM,243687,20,Alumier Labs Inc.,,15
3,Aug 8 2022 2:20PM,243690,10,ISDIN Corporation,,1
4,Aug 8 2022 2:17PM,243688,10,ISDIN Corporation,,9
5,Aug 8 2022 2:13PM,243686,19,"NAPP Technologies, LLC",,1
6,Aug 8 2022 2:06PM,243685,20,"Exact-Rx, Inc.",,4
7,Aug 8 2022 2:06PM,243684,20,"Exact-Rx, Inc.",,17
8,Aug 8 2022 1:53PM,243683,19,"AdvaGen Pharma, Ltd",1,
9,Aug 8 2022 1:41PM,243682,10,"Citieffe, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 2:27PM,243689,20,Alumier Labs Inc.,34,
1,Aug 8 2022 2:26PM,243692,19,"AdvaGen Pharma, Ltd",3,
2,Aug 8 2022 2:24PM,243687,20,Alumier Labs Inc.,15,
3,Aug 8 2022 2:20PM,243690,10,ISDIN Corporation,1,
4,Aug 8 2022 2:17PM,243688,10,ISDIN Corporation,9,
5,Aug 8 2022 2:13PM,243686,19,"NAPP Technologies, LLC",1,
6,Aug 8 2022 2:06PM,243685,20,"Exact-Rx, Inc.",4,
7,Aug 8 2022 2:06PM,243684,20,"Exact-Rx, Inc.",17,
8,Aug 8 2022 1:53PM,243683,19,"AdvaGen Pharma, Ltd",,1
9,Aug 8 2022 1:41PM,243682,10,"Citieffe, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 2:27PM,243689,20,Alumier Labs Inc.,34,
1,Aug 8 2022 2:26PM,243692,19,"AdvaGen Pharma, Ltd",3,
2,Aug 8 2022 2:24PM,243687,20,Alumier Labs Inc.,15,
3,Aug 8 2022 2:20PM,243690,10,ISDIN Corporation,1,
4,Aug 8 2022 2:17PM,243688,10,ISDIN Corporation,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 2:27PM,243689,20,Alumier Labs Inc.,34.0,NaN
1,Aug 8 2022 2:26PM,243692,19,"AdvaGen Pharma, Ltd",3.0,NaN
2,Aug 8 2022 2:24PM,243687,20,Alumier Labs Inc.,15.0,NaN
3,Aug 8 2022 2:20PM,243690,10,ISDIN Corporation,1.0,NaN
4,Aug 8 2022 2:17PM,243688,10,ISDIN Corporation,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 2:27PM,243689,20,Alumier Labs Inc.,34.0,0.0
1,Aug 8 2022 2:26PM,243692,19,"AdvaGen Pharma, Ltd",3.0,0.0
2,Aug 8 2022 2:24PM,243687,20,Alumier Labs Inc.,15.0,0.0
3,Aug 8 2022 2:20PM,243690,10,ISDIN Corporation,1.0,0.0
4,Aug 8 2022 2:17PM,243688,10,ISDIN Corporation,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2924038,21.0,2.0
16,243617,1.0,0.0
19,1218354,5.0,2.0
20,1218413,70.0,1.0
21,487323,2.0,0.0
22,243627,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2924038,21.0,2.0
1,16,243617,1.0,0.0
2,19,1218354,5.0,2.0
3,20,1218413,70.0,1.0
4,21,487323,2.0,0.0
5,22,243627,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,21.0,2.0
1,16,1.0,0.0
2,19,5.0,2.0
3,20,70.0,1.0
4,21,2.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,16,Released,1.0
2,19,Released,5.0
3,20,Released,70.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Executing,2.0,0.0,2.0,1.0,0.0,0.0
Released,21.0,1.0,5.0,70.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Executing,2.0,0.0,2.0,1.0,0.0,0.0
1,Released,21.0,1.0,5.0,70.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Executing,2.0,0.0,2.0,1.0,0.0,0.0
1,Released,21.0,1.0,5.0,70.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()